# Flavors of Cacao - Análisis de visualización

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs # load make_blobs to simulate data
from sklearn import decomposition # load decomposition to do PCA analysis with sklearn

Vamos a trabajar con un set de datos sobre un ranking de barras de chocolate.

Lo primero de todo es cargar con el dataset y para eso necesitamos parsear CSV ya que la información viene con este formato.

In [ ]:
chocolate_data = pd.read_csv("../input/flavors_of_cacao.csv")

Para trabajar con este dataset primero vamos a transformar un par de datos para que sea más fácil trabajar en el futuro con ellos.

Primero vamos a modificamos el nombre de las columnas para que sea más fácil trabajar con ellas, quitando espacios y nombres muy largos.

In [ ]:
original_col = chocolate_data.columns
new_col = ['Company', 'Species', 'REF', 'ReviewDate', 'CocoaPercent','CompanyLocation', 'Rating', 'BeanType', 'Country']
chocolate_data =chocolate_data.rename(columns=dict(zip(original_col, 
new_col)))
chocolate_data.head()

Luego vamos a transformar los porcentajes de la columna **CocoaPercent**  a valores entre **0** y **1** para que sea más fácil poder graficar/comparar estos valores.

In [ ]:
#Remove % sign from CocoaPercent column 
chocolate_data['CocoaPercent'] = chocolate_data['CocoaPercent'].str.replace('%','').astype(float)/100
chocolate_data.head()

En los siguientes graficos podemos ver la distribución de valores que tienen las Columnas **Rating**, **REF** y **CocoaPercent**

Como podemos ver en el primer gráfico el **Rating** varía desde **1** hasta **5** pero la mayoría de los chocolates están entre los valores de **2,5** y **4** de **Rating**

En el caso de **REF** el cual son los números de referencia de los datos agregados. Mientras más alto el **REF** más nuevo el dato.

Por último como podemos ver el en el tercer gráfico, el **CocoaPercent** varía entre **0.4** y **1** (**40%** y **100%** ya que nosotros modificamos esta columna). Acá podemos visualizar que claramente que las companias tienen una tendencia a fabricar chocolates con un ** 70%** de Cacao.

In [ ]:
sb.distplot(chocolate_data['Rating'],kde = False)
plt.show()
sb.distplot(chocolate_data['REF'],kde = False)
plt.show()
sb.distplot(chocolate_data['CocoaPercent'],kde = False)
plt.show()

En los siguientes gráficos se intenta visualizar si hay relación entre ciertas columnas.

En el primer gráfico vemos que no hay mucha relación entre  el porcentaje de cacao y el rating ya que la mayor parte de los datos están agrupados en el centro con algunos casos dispersos.

Un caso parecido pasa con el segundo gráfico. Lo que podríamos decir es que se fueron "normalizando" los ratings a lo largo del tiempo  entre los valores  **2** y **4**

In [ ]:
ax = plt.axes()
ax.scatter(chocolate_data['CocoaPercent'], chocolate_data['Rating'])
ax.set(xlabel='Cocoa Strength',
       ylabel='Rating',
       title='Cocoa Strength vs Rating')

In [ ]:
ax = plt.axes()
ax.scatter(chocolate_data['ReviewDate'], chocolate_data['Rating'])
ax.set(xlabel='ReviewDate',
       ylabel='Rating',
       title='Country vs Rating')

En este gráfico de barras podemos visualizar las 5 compañias que más aparecen en el dataset siendo *Soma* la que ha liderado el ranking.

In [ ]:
bc = plt.axes()

Companyfreq=chocolate_data['Company'].value_counts()
x=[] #init empty lists
y=[]
for i in range (0,5):
    x.append(Companyfreq.axes[0][i])
    y.append(Companyfreq[i])
    
bc.bar(x,y)

Por otro lado podemos agrupar datos de manera que podemos sacar la media del rating de cada pais y así poder graficar un ranking de los paises que son los mejores productores de barras de chocolate segun este dataset.

In [ ]:
mean_by_country = chocolate_data.groupby(["CompanyLocation"])['Rating'].mean()
mean_sorted = mean_by_country.sort_values(ascending=False)
top_bottom_5 = pd.concat([mean_sorted[:5], mean_sorted[-5:]])
top_bottom_5.plot('barh')

Podemos definir un sistema de rating que sea

*  Raiting < 3.0  => Insatisfactorio
*  Raiting >= 3.0 Y Raiting < 4.0  =>  Satisfactorio
*  Raiting >= 4.0  => Premium

Como podemos ver en el gráfico un **5.6%** de las barras de chocolate se las conciera como Premium

In [ ]:
unsatisfactory = chocolate_data[chocolate_data['Rating'] < 3.0] 
satisfactory = chocolate_data[(chocolate_data['Rating'] >= 3.0) & (chocolate_data['Rating'] < 4.0)] 
pre_elite = chocolate_data[chocolate_data['Rating'] >= 4.0] 
label_names=['Insatisfactorio','Satisfactorio','Premium']

sizes = [unsatisfactory.shape[0],satisfactory.shape[0],pre_elite.shape[0]]
explode = (0.05,0.05,0.05)
plt.pie(sizes,labels=label_names,explode=explode,autopct='%1.1f%%',pctdistance=0.85
        ,startangle=90,shadow=True)
fig=plt.gcf()
my_circle=plt.Circle((0,0),0.7,color='white') #white center
fig.gca().add_artist(my_circle)
plt.axis('equal')
plt.tight_layout()
plt.show()

**===================================================================================================================================================**

# Principal Component Analysis (PCA)

Trabajar con dataset muy grandes trae muchos inconvenientes no solo en cuestión de tiempo de procesamiento sino tambien a la hora de graficar.

El análisis de componentes principales, o **PCA**, es una técnica estadística para convertir datos de alta a baja dimensionalidad. Esto se realiza al seleccionar las características más importantes que capturan la máxima información sobre el conjunto de datos. Las características se seleccionan en base a la varianza que causan en la salida. La característica que causa la mayor variación se la llama componente principal. La característica que es responsable de la segunda varianza más alta se la llama segundo componente principal, y así sucesivamente. Es importante mencionar que los componentes principales no tienen ninguna correlación entre sí.

Para este caso vamos a simular el dataset usando el modulo **make_blobs** de la librerpia **scikit-learn** como tambien el modulo de **PCA**  que ya importarmos al principio.

El modulo **make_blobs** sirve para contruir datasets simulados, sirve facilmente para contruir multiples clusters gaussianos y es muy utlizado para testear algoritmos de clustering.
En este caso vamos a construir una matriz de 100x10 (100 muestras con 10 observaciones). Estas 100 muestras fueron generadas por 4 clusters diferentes. Ya que estamos simulando la información nosotros sabemos  a que cluster pertenece cada muestra.

In [ ]:
X1, Y1 = make_blobs(n_features=10, 
         n_samples=100,
         centers=4, random_state=4,
         cluster_std=2)
print(X1.shape)

**X1** es la matriz de 100x10 e **Y1** es la asginación de cada cluster. 

El siguiente paso es crear el modelo de **PCA** con **4** componentes.

In [ ]:
pca = decomposition.PCA(n_components=4)

Ya que no tenemos que hacer transformaciónes sobre los datos, pasamos al siguiente paso donde ejecutamos la etapa de entrenamiento con la matriz **X1** y obtenemos nuestras componentes principales.

In [ ]:
pc = pca.fit_transform(X1)

Ahora volvemos a genera la matriz incluyendo el vector de clusters previamente creador por la librería.

In [ ]:
pc_df = pd.DataFrame(data = pc , 
        columns = ['PC1', 'PC2','PC3','PC4'])
pc_df['Cluster'] = Y1
pc_df.head()

Si examinamos la varianza de cada componente obtenida podemos notar que el las primeras **2** componentes abarcan el **70%** de la variación del dataset

In [ ]:
print(pca.explained_variance_ratio_)
print(str(round((pca.explained_variance_ratio_[0] + pca.explained_variance_ratio_[1])*100,2))+"%")

Acá podemos ver la varianza individual de cada componente

In [ ]:
df = pd.DataFrame({'var':pca.explained_variance_ratio_,
             'PC':['PC1','PC2','PC3','PC4']})
sb.barplot(x='PC',y="var", 
           data=df, color="c");

Ahora podemos usar las primeras **2** componentes principales (las que tienen una varianza mayor) para realizar un scatter plot.

In [ ]:
sb.lmplot( x="PC1", y="PC2",
  data=pc_df, 
  fit_reg=False, 
  hue='Cluster', # color by cluster
  legend=True,
  scatter_kws={"s": 80}) # specify the point size

Podemos ver claramente los **4** clusters en nuestros datos, solo necesitamos de las **2** primeras componentes para separar completamente nuestros clusters.

**===================================================================================================================================================**

# Algoritmos genéticos

## Teoría básica
Un algoritmo genético se inspira en la teoría de Darwin de la selección Natural para resolver problemas de optimización. Es una buena solución sobre todo cuando se tiene información incompleta o imperfecta, o incluso limitada capacidad computacional.
Los tres principios fundamentales necesarios para la evolución (según la Teoría de Darwin)  a suceder son: 

   1.  **Herencia:**  Debe ser un proceso por el cual los niños reciben la propiedad de sus padres.
   2. **Variación:** Debe haber una variedad de rasgos presentes en la población o un medio de introducir una variación. 
   3. **Selección:** Debe existir un mecanismo por el que algunos miembros de la población puedan ser padres y transmitir su información genética y otros no (supervivencia del más apto).
  
## Como llevar los principios a un algoritmo genético

Hay 5 etapas en un algoritmo genético:
1. Crear una población inicial
2. Definir una función de fitness
3. Seleccionar los padres
4. Realizar un crossover (cruce)
5. Realizar una mutación


### Código

A continuación vamos a mostrar un código en Python que realiza un algoritmo genético muy básico a modo de ejemplo en el cual se trata de reproducir/copiar una cadena de caracteres (el target en este caso) sin realmente usar la cadena más que para comparar.

La idea general de este algoritmo genético es un bucle que utiliza las funciones anteriores para generar una secuencia de gen candidato, compararla con la mejor opción anterior y mutarla al azar hasta que todos los genes coincidan con los del objetivo.

Hay muchas formas de calcular un valor de fitness (qué tan cerca está la cadena del objetivo) para la cadena generada. Para este problema en particular, simplemente contaremos el número de caracteres que son iguales entre la cadena candidata y la cadena de destino (target).

In [ ]:
import datetime
import random

#Definición de constantes
geneSet = "abcdefghijklmnñopqrstuvwxyzABCDEFGHIJKLMNÑOPQRSTUVWXYZ "
target = "Hola Mundo"

#Inicialización de variables
random.seed(2)
startTime = datetime.datetime.now()

def generate_parent(lenght):
    """
        Define muestra aleatoria para que sea nuestro padre
        Ejemplo: random.sample(geneSet, 5) == > ['o', 'S', 'D', 'B', 'L']

    """
    genes = []
    while len(genes) < lenght:
        sampleSize = min(lenght - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)

def get_fitness(guess):
    """
        Función de aptitud donde sumamos 1 si nuestra muestra aleatoria
        coincide en lugar y caracter de nuestro target
        Ejemplo: 
        zip(target, "Ho iLEPHIZ") sería
        [('H', 'H'),
         ('o', 'o'),
         ('l', ' '),
         ('a', 'i'),
         (' ', 'L'),
         ('M', 'E'),
         ('u', 'P'),
         ('n', 'H'),
         ('d', 'I'),
         ('o', 'Z')]
         
         al recorrerlo  la primera iteración expected = H y actual = H por lo tanto suma 1
         .
         .
         .
         en la ultima iteración expected = o y actual = Z por lo tanto suma 0
         resultado es 2 por las 2 primeras iteraciones
    """
    return sum(1 for expected, actual in zip(target, guess) if expected == actual)

def mutate(parent):
    """
        Creamos una muestra aletoria y la vamos a agregar
    """
    index = random.randrange(0, len(parent))
    childGenes = list(parent)
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate if newGene == childGenes[index] else newGene
    return ''.join(childGenes)

def display(guess):
    timeDiff =  datetime.datetime.now() - startTime
    fitness = get_fitness(guess)
    print('{}\t{}\t{}'.format(guess, fitness, timeDiff))
    
bestParent = generate_parent(len(target)) # Genera String aleatorio de la misma longitud que el target
bestFitness = get_fitness(bestParent)
display(bestParent)

while True:
    child = mutate(bestParent)
    childFitness = get_fitness(child)
    if bestFitness >= childFitness:
        continue
    display(child)
    if childFitness >= len(bestParent):
        break
    bestFitness = childFitness
    bestParent = child